# Latex to Excel
Ce notebook permet de réaliser la conversion des tableaux depuis le code latex jusqu’à un format excel

In [1]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font


In [2]:
# Perimetre fonctionnel

Nous extrayons manuellement le tableau du fichier latex.

In [3]:
data = """
\hline  Ref.	\cellcolor{gray!25}&Type \cellcolor{gray!25}& Nom\cellcolor{gray!25}& Descriptif \cellcolor{gray!25}& \bf Estimation \cellcolor{gray!25} & \bf Complexité \cellcolor{gray!25} & \bf Priorité \cellcolor{gray!25} & \bf V1 \cellcolor{gray!25} & \bf (V2) \cellcolor{gray!25}\\
\hline  	\bf A\cellcolor{gray!50}& \bf Écran\cellcolor{gray!50}& \bf Intro\cellcolor{gray!50}& \bf Écran de lancement de l’application\cellcolor{gray!50}& 4500 €\cellcolor{gray!50} & \cellcolor{gray!50} &  \cellcolor{gray!50} & \cellcolor{gray!50} &  \cellcolor{gray!50}\\
\hline  	\bf A1& Fonctionnalité & UX & Demander l’autorisation de démarrer la vidéo et le son & 750 & 1 &  2 &  & \bf X\\
\hline  	\bf A2& Fonctionnalité & Responsive & Adaptation à toutes les tailles d’écran & 750 & 3 &  5 & \bf X & \\
\hline  	\bf A3& Fonctionnalité & Web Design & Couleur, logo et design appartenant à la charte graphique & 750 & 3 &  4 & \bf X & \\
\hline  	\bf A4& Fonctionnalité & Security & Bouton de création, login et interface de connexion sécurisée, à son compte utilisateur & 750 & 5 &  4 &  & \bf X\\
\hline  	\bf A5& Fonctionnalité & Https & Connexion chiffrée pour les visiteurs & 750 & 5 &  4 & \bf X & \\
\hline  	\bf A6& Fonctionnalité & Cookie & Demande de l’accord pour le stockage de données personnelles (traitements - RGPD) & 750 & 2 & 5 &  & \bf X \\
\hline  	\bf B\cellcolor{gray!50}& \bf CRUD\cellcolor{gray!50}& \bf Parameters\cellcolor{gray!50}& \bf Interface de configuration\cellcolor{gray!50}& 7500 €\cellcolor{gray!50} & \cellcolor{gray!50} &  \cellcolor{gray!50} & \cellcolor{gray!50} &  \cellcolor{gray!50}\\
\hline  	\bf B1& Fonctionnalité & Parameters & Gestion des options du comportement de l’application & 1875 & 5 &  4 & \bf X & \\
\hline  	\bf B2& Fonctionnalité & Changement & Modification des informations personnelles & 1875 & 5 &  4 & \bf X & \\
\hline  	\bf B3& Fonctionnalité & Notation & Sélection des historiques de navigation retenus par le système de recommandation & 1875 & 5 & 4 &  & \bf X \\
\hline  	\bf B4& Fonctionnalité & Hash/Salage & Stockage des mots de passe hacher & 1875 & 1 & 4 &  & \bf X \\
\hline  	\bf C\cellcolor{gray!50}& \bf Accessibility\cellcolor{gray!50}& \bf W3C\cellcolor{gray!50}& \bf Adaptation aux handicaps\cellcolor{gray!50}& 1500 €\cellcolor{gray!50} & \cellcolor{gray!50} &  \cellcolor{gray!50} & \cellcolor{gray!50} &  \cellcolor{gray!50}\\
\hline  	\bf C1& Fonctionnalité & Mal-entendant & Écriture des sous-titres des vidéos et descriptif des scènes & 500 & 1 &  3 &  & \bf X\\
\hline  	\bf C2& Fonctionnalité & Déficience visuelle & Mode contraste élevé & 500 & 1 &  4 &  & \bf X \\
\hline  	\bf C3& Fonctionnalité & Non-voyant & Description optionnelle des médias & 500 & 1 &  3 & \bf X & \\
\hline  	\bf D\cellcolor{gray!50}& \bf Translation\cellcolor{gray!50}& \bf Living in EU\cellcolor{gray!50}& \bf Internationalisation\cellcolor{gray!50}& 2000 € \cellcolor{gray!50} & \cellcolor{gray!50} &  \cellcolor{gray!50} & \cellcolor{gray!50} &  \cellcolor{gray!50}\\
\hline  	\bf D1& Fonctionnalité & Export & Gestion du mode multi-nationalité & 1000 & 2 &  2 &  & \bf X \\
\hline  	\bf D2& Fonctionnalité & Etna & Gestion de la langue Italienne & 1000 & 2 &  5 & \bf X & \\
\hline  	\bf E\cellcolor{gray!50}& \bf Deep Learning\cellcolor{gray!50}& \bf Open AI \cellcolor{gray!50}& \bf Data Science et IA\cellcolor{gray!50}& 10000 €\cellcolor{gray!50} & \cellcolor{gray!50} &  \cellcolor{gray!50} & \cellcolor{gray!50} &  \cellcolor{gray!50}\\
\hline  	\bf E1& Fonctionnalité & Nettoyage & Préparation des données pour le DL& 1000 & 1 &  5 & \bf X &  \\
\hline  	\bf E2& Fonctionnalité & EDA & Orienter la lecture des data sur l’écartement des biais présents dans la base de données & 1500 & 5 &  1 & \bf X &  \\
\hline  	\bf E3& Fonctionnalité & Modélisation & Entraînement du modèle & 1500 & 1 &  5 & \bf X &  \\
\hline  	\bf E4& Fonctionnalité & Test & Test du modèle & 1500 & 1 &  5 & \bf X &  \\
\hline  	\bf E5& Fonctionnalité & Interprétation & Interprétation du modèle & 1500 & 3 &  5 &  & \bf X \\
\hline  	\bf E6& Fonctionnalité & Dev/Ops & Conservation du code sur GitHub & 1500 & 1 &  3 & \bf X & \\
\hline  	\bf E7& Fonctionnalité & Éthique & Éditer un communiqué sur l’éthique du modèle & 1500 & 3 &  2 &  & \bf X \\
\hline  	\bf F\cellcolor{gray!50}& \bf UpLoad\cellcolor{gray!50}& \bf Recollect \cellcolor{gray!50}& \bf Inférence sur nouvelles données\cellcolor{gray!50}& 3000 €\cellcolor{gray!50} & \cellcolor{gray!50} &  \cellcolor{gray!50} & \cellcolor{gray!50} &  \cellcolor{gray!50}\\
\hline  	\bf F1& Fonctionnalité & Stockage & Transmission chiffrée et sauvegarde en blob des photos (> 800Ko, <12Mo par photos) & 750 & 2 & 5 & \bf X &  \\
\hline  	\bf F2& Fonctionnalité & Table Relationnelle & Transmission (chiffrée) et sauvegarde des noms des photos et de leurs références (stats) & 750 & 2 & 5 & \bf X &  \\
\hline  	\bf F3& Fonctionnalité & Resolution & Retouche pics en vue d’utilisation par le modèle (harmonisation de la résolution/format) & 750 & 2 & 4 & \bf X &  \\
\hline  	\bf F4& Fonctionnalité & Backup & Duplication de la base de données sur deux régions, au minimum & 750 & 3 & 3 &  & \bf X \\
\hline  	\bf G\cellcolor{gray!50}& \bf Deployment\cellcolor{gray!50}& \bf Cloud \cellcolor{gray!50}& \bf Disponibilité du service\cellcolor{gray!50}& 10000 €\cellcolor{gray!50} & \cellcolor{gray!50} &  \cellcolor{gray!50} & \cellcolor{gray!50} &  \cellcolor{gray!50}\\
\hline  	\bf G1& Fonctionnalité & PyTest & Test de Déploiement en Dev/Ops GitHub Action & 2000 & 4 & 5 & \bf X &  \\
\hline  	\bf G1& Fonctionnalité & Kubernetes & Déploiement en pod avec Kubernetes & 2000 & 4 & 5 & \bf X &  \\
\hline  	\bf G2& Fonctionnalité & Pods & Connexion des pods d’application et de base de données & 2000 & 3 & 5 & \bf X &  \\
\hline  	\bf G3& Fonctionnalité & Redondance du service & Déploiement au plus près des clients & 2000 & 1 & 5 &  & \bf X \\
\hline  	\bf G4& Fonctionnalité & Adaptation & Ajustement du nombre de serveur en fonction du nombre d’utilisateur & 2000 & 2 & 5 & & \bf X \\
\hline  	\bf G5& Fonctionnalité & Monitoring & Remonté d’alerte au problème à la moindre prédiction, connexion, ou téléversement & 2000 & 2 & 5 & & \bf X \\
"""

Nous remplaçons quelques éléments de formatage.

In [4]:
d = data.replace('\\cellcolor{gray!50}', '')\
.replace('\\cellcolor{gray!15}', '')\
.replace('\\cellcolor{gray!25}', '')\
.replace('\bf', ' ')\
.replace('\\dots', '...')\
.replace('\\hline', '').replace('\n',' ')\
.replace('\t', ' ').replace('\\', ' \n ').replace('&', ' \t ')


In [5]:
d.encode('utf-8').decode()

'   Ref.  \t Type  \t  Nom \t  Descriptif  \t    Estimation   \t    Complexité   \t    Priorité   \t    V1   \t    (V2)  \n       A \t    Écran \t    Intro \t    Écran de lancement de l’application \t  4500 €  \t    \t     \t    \t    \n       A1 \t  Fonctionnalité  \t  UX  \t  Demander l’autorisation de démarrer la vidéo et le son  \t  750  \t  1  \t   2  \t    \t    X \n       A2 \t  Fonctionnalité  \t  Responsive  \t  Adaptation à toutes les tailles d’écran  \t  750  \t  3  \t   5  \t    X  \t   \n       A3 \t  Fonctionnalité  \t  Web Design  \t  Couleur, logo et design appartenant à la charte graphique  \t  750  \t  3  \t   4  \t    X  \t   \n       A4 \t  Fonctionnalité  \t  Security  \t  Bouton de création, login et interface de connexion sécurisée, à son compte utilisateur  \t  750  \t  5  \t   4  \t    \t    X \n       A5 \t  Fonctionnalité  \t  Https  \t  Connexion chiffrée pour les visiteurs  \t  750  \t  5  \t   4  \t    X  \t   \n       A6 \t  Fonctionnalité  \t  Cookie  \t

In [6]:
with open('per_funct.csv', 'w') as f:
    f.writelines(d.encode('utf-8').decode())

Nous enregistrons le texte sous forme de tableur excel 2010 "xlsx", afin de pouvoir utiliser la bibliothèque openpyxl.

In [7]:
pd.read_csv('per_funct.csv', sep='\t').to_excel('per_funct.xlsx')

In [8]:
wb = load_workbook(filename = 'per_funct.xlsx')
sheet_ranges = wb['Sheet1']
print(sheet_ranges['B3'].value)

       A1 


Nous créons les couleurs de fond que nous utiliserons dans le fichier

In [9]:
grayLightFill = PatternFill(start_color='e0dede',
                   end_color='e0dede',
                   fill_type='solid')
grayDarkFill = PatternFill(start_color='a7a7a7',
                   end_color='a7a7a7',
                   fill_type='solid')
blackFill = PatternFill(start_color='404040',
                   end_color='404040',
                   fill_type='solid')

Nous appliquons le formatage.

In [10]:
for cell in sheet_ranges['B1:J1']:
    for x in range(len(cell)):
        cell[x].fill = blackFill
        cell[x].font = Font(color="FFFFFF", bold=True) 
n = 2
for cell1 in sheet_ranges['B2:B40']:
    text = str(cell1[0].value).rstrip().lstrip()
    if text.isalpha():
        for cell in sheet_ranges['B' + str(n) + ':J' + str(n)]:
            for x in range(len(cell)):
                cell[x].fill = grayDarkFill
    else:
        for cell in sheet_ranges['B' + str(n) + ':J' + str(n)]:
            for x in range(len(cell)):
                cell[x].fill = grayLightFill
    n += 1


In [11]:
# Code pour détruire la colonne d’index supplémentaire issue du cadre de données.
sheet_ranges.delete_cols(1, 1)
sheet_ranges.title = "Perimetre_functionnel"

In [12]:
wb.save(filename = 'dest_functional_perimeter.xlsx')

# Color Format for BackLog
Nous répétons l’opération pour le tableau suivant.

In [13]:
data = """
\hline  Titre	\cellcolor{gray!50}&En tant que \cellcolor{gray!50}& Je veux\dots\cellcolor{gray!50}&  \dots afin de\dots\cellcolor{gray!50}& \bf Pondération\cellcolor{gray!50} & \bf Données nécessaires\cellcolor{gray!50} \\
\hline  Connexion via une adresse mail& 
Utilisateur de l'application &
  pouvoir me connecter à mon compte sur l'application mobile en utilisant une adresse mail& je peux voir les informations me concernant (infos personnelles, photos prises, articles suggérés, ...)& 
  A définir en relatif par rapport aux autres user stories en donnant un chiffre parmi : 1, 3, 5, 8, 13, 20, 50, 100.&
{•} adresse mail utilisateur

{•}  mot de passe 
\\
\hline   Conserver le mot de passe haché \cellcolor{gray!15} & Administrateur de Base de données \cellcolor{gray!15}& Ne pas conserver le mot de passe en clair \cellcolor{gray!15}& 
Réduire le risque de vol d’identité en cas de piratage  \cellcolor{gray!15}& 
L’utilisation d’un salage et d’une graine lors du hachage augmente la sécurité \cellcolor{gray!15}& mot de passe, sel, graine\cellcolor{gray!15}\\
\hline   Flouter les visages des utilisateurs & Data Scientist / Utilisateur de l’application & Ne pas conserver l’image pouvant servir à la reconnaissance faciale & Ne pas utiliser cette données, car inutile pour le traitement & Les données d’entraînement peuvent être transformer identiquement & Récolte d’éléments vestimentaires avec visage flouté\\
\hline   Création de recommandation d’achat à partir des cliques \cellcolor{gray!15}& UXDesigner / Utilisateur de l’application \cellcolor{gray!15}& Conserver des historiques de navigations pour faire du neuroprofilage  \cellcolor{gray!15}& Proposer les articles susceptibles de l’intéresser sur la base de son historique de navigation.  \cellcolor{gray!15}& Le client a un droit de sélectionner ce qu’il veut que la plateforme conserve comme données d’historique de navigation \cellcolor{gray!15}& L’historique de navigation, relié au pseudo du client. \cellcolor{gray!15}\\
\hline   Création d’un panier d’achat & UXDesigner / Utilisateur de l’application & Conserver une liste d’achats souhaités n & Valider son panier après la phase shopping, au moment du passage en caisse & Ce sont des données peu sensible, modifiable à tout moment par l’utilisateur, mais conserver de manière illimité, comme un service pour l’utilisateur. & Sélection d’article lié au pseudo de l’utilisateur \\
\hline   Création d’une liste d’envie \cellcolor{gray!15}& UXDesigner / Utilisateur de l’application \cellcolor{gray!15}& Conserver une liste d’envie \cellcolor{gray!15}& Garder des idées d’achats pour une occasion à horizon plus lointaine \cellcolor{gray!15}& Ce sont des données peu sensible, modifiable à tout moment par l’utilisateur, mais conserver de manière illimité, comme un service pour l’utilisateur. \cellcolor{gray!15}& Sélection d’article lié au pseudo de l’utilisateur \cellcolor{gray!15}\\
\hline   Système de notation des prédictions & Data Scientist / Utilisateur de l’application & L’utilisateur donne une note ou un like/dislike ou un swipe sur les vêtements proposés & Améliorer les prédictions en connaissant mieux les goûts des utilisateurs (Matrice Users/Items) & Durée de stockage définie par l’utilisateur sur 0, 1, 2 ou 3 ans. & Pseudo utilisateur, Articles, Notes \\
\hline   Système d’association de produit \cellcolor{gray!15}& Data Scientist / Utilisateur de l’application \cellcolor{gray!15}& L’utilisateur reçoit des recommandations d’articles similaire à ceux qui sont prédits \cellcolor{gray!15}& Augmenter le nombre d’articles similaire (Matrice Items/Items) \cellcolor{gray!15}& La base de données des articles en vente doit pouvoir être catégorisé en hiérarchie de produit. \cellcolor{gray!15}& Les informations sur les produits, photos, descriptifs. \cellcolor{gray!15}\\
\hline   Création d’une interface & Web Design / Utilisateur de l’application & Une interface intuitive avec une charte graphique de la marque & Offrir une bonne expérience utilisateur & Les cookies doivent être autorisées par l’utilisateur & Cookies, IP, Browser fingerprints, stockés maximum 1 an si l’utilisateur autorise les CGU. \\
\hline   Détection d’une connexion depuis un nouvel appareil \cellcolor{gray!15}& Security / Utilisateur de l’application \cellcolor{gray!15}& Sécuriser l’application pour l’utilisateur\cellcolor{gray!15} & Éviter / Alerter en cas de piratage \cellcolor{gray!15}& Les données ne concerne pas l’identité de la personne directement mais les supports numériques utilisés pour se connecter. \cellcolor{gray!15}& Les données conservés sont l’adresse Mac de la machine, le nom du système OS, le type de navigateur web. \cellcolor{gray!15}\\
"""

In [14]:
d = data.replace('\\cellcolor{gray!50}', '')\
.replace('\\cellcolor{gray!15}', '')\
.replace('\\cellcolor{gray!25}', '')\
.replace('\bf', ' ')\
.replace('\\dots', '...')\
.replace('\\hline', '').replace('\n',' ')\
.replace('\t', ' ').replace('\\', ' \n ').replace('&', ' \t ')


In [15]:
d.encode('utf-8').decode()

"   Titre  \t En tant que  \t  Je veux... \t   ... afin de... \t    Pondération  \t    Données nécessaires  \n    Connexion via une adresse mail \t   Utilisateur de l'application  \t    pouvoir me connecter à mon compte sur l'application mobile en utilisant une adresse mail \t  je peux voir les informations me concernant (infos personnelles, photos prises, articles suggérés, ...) \t     A définir en relatif par rapport aux autres user stories en donnant un chiffre parmi : 1, 3, 5, 8, 13, 20, 50, 100. \t  {•} adresse mail utilisateur  {•}  mot de passe   \n     Conserver le mot de passe haché   \t  Administrateur de Base de données  \t  Ne pas conserver le mot de passe en clair  \t   Réduire le risque de vol d’identité en cas de piratage   \t   L’utilisation d’un salage et d’une graine lors du hachage augmente la sécurité  \t  mot de passe, sel, graine \n     Flouter les visages des utilisateurs  \t  Data Scientist / Utilisateur de l’application  \t  Ne pas conserver l’image pouvant ser

In [16]:
with open('backlog.csv', 'w') as f:
    f.writelines(d.encode('utf-8').decode())

In [17]:
pd.read_csv('backlog.csv', sep='\t').to_excel('backlog.xlsx')

In [18]:
from openpyxl import load_workbook
wb_backlog = load_workbook(filename = 'backlog.xlsx')
ws_backlog = wb_backlog['Sheet1']
print(ws_backlog['B3'].value)

     Conserver le mot de passe haché   


In [19]:
ws_backlog.delete_cols(1, 1)

In [20]:
for cell in ws_backlog['A1:F1']:
    for x in range(len(cell)):
        cell[x].fill = blackFill
        cell[x].font = Font(color="FFFFFF", bold=True) 
n = 2
for cell1 in ws_backlog['A2:A11']:
    text = str(cell1[0].value).rstrip().lstrip()
    if n % 2 != 0:
        for cell in ws_backlog['A' + str(n) + ':F' + str(n)]:
            for x in range(len(cell)):
                cell[x].fill = grayLightFill
    n += 1

In [21]:
ws_backlog.title = "Backlog"

wb_backlog.save(filename = 'dest_backlog.xlsx')

In [22]:
xl1 = load_workbook('dest_functional_perimeter.xlsx')
# sheet you want to copy
s = load_workbook('dest_backlog.xlsx').active
s._parent = xl1
xl1._add_sheet(s)
xl1.save('backlog_and_functional_perimeter.xlsx')

# Color Format for Plan d’action contre les risques
Nous répétons l’opération pour le tableau suivant.

In [23]:
data = """
\hline 
 \bf Resp.  & \bf Accident & \bf Probables Causes & \parbox{1.5cm}{\bf Initial Risk Probabilité} & \bf Sévérité & \bf Risk Level & \bf Actions Preventives& \bf Reparations & \parbox{1.5cm}{\bf Residual Risk Probabilité} &\bf Sévérité & \bf Risk Level \\
\hline
 Codeur  & Usurpation d’identité & Stockage en clair du password& 3 & 4 & \cellcolor{red!50} 12 & Fonction de dérivation, graine et salage du mot de passage & Alerte client pour toute nouvelle connexion IP & 1 & 2 & \cellcolor{green!50} 2 \\ 
  Codeur  & Injection & Données hostiles & 4 & 4 & \cellcolor{red!75} 16 & Échappement des chars spéciaux, utilisation de LIMIT en SQL & Stocker données nécessaires, seulement & 3 & 4 & \cellcolor{yellow!50} 12 \\ 
  Codeur  & Déni de service & Fichier accessible par tous via lien & 5 & 2 & \cellcolor{yellow!50} 10 &  Tester la performance E/S & Réarchitecturer, optimiser, mettre en cache & 4 & 2 & \cellcolor{yellow!50} 8 \\ 
Data Scientist & Intégrité des données & Data sérialisées non sécurisées & 3 & 4 & \cellcolor{red!50} 12 &  Chiffrer et signer les data échangées & Ségrégation du pipeline, contrôler l’accès au code & 2 & 3 & \cellcolor{yellow!50} 6 \\ 
Dev/Ops & Security Misconfiguration & Un composant est obsolète ou vulnérable & 3 & 3.5 & \cellcolor{red!50} 10.5 & Ne pas installer des fonctionnalités inutilisées & Utiliser la conteneurisation, une architecture segmentée sûre & 3 & 2 & \cellcolor{yellow!50} 6 \\ 
Codeur & Composant Vulnérable & Pas de veille sur failles de sécurité & 2 & 3,5 & \cellcolor{yellow!50} 7 & Supprimer les dépendances inutiles & Mettre en dépôt une version risquée, lue et reconnue & 1 & 3.5 & \cellcolor{green!50} 3.5 \\ 
Codeur & Erreur Gestion Mémoire & Mauvaise utilisation mémoire & 5 & 3 & \cellcolor{red!50} 15 & Typer les variables, design pattern & Envisager le codage en Go ou Rust & 1 & 3 & \cellcolor{green!50} 3 \\ 
Codeur & Authentification & Autoriser les comptes par défaut & 3.6 & 5 & \cellcolor{red!75} 18 & Livrer, déployer sans compte par défaut & Intégrer des tests de password à la création, au changement & 3 & 3 & \cellcolor{yellow!50} 9 \\ 
Dev/Ops & Vol de données bancaires & Mémorisation du numéro de CB & 2 & 8 & \cellcolor{red!75} 16 & Utiliser une tokenisation de norme sûre comme PCI DSS & Classifier, identifier ou rejeter les données sensibles & 2 & 4 & \cellcolor{yellow!50} 8 \\ 
Data Scientist & SR connaissant peu les clients & Connaissance des clients faible & 5 & 3 & \cellcolor{red!75} 15 & Utiliser la reconnaissance d’image pour les recommandations & Enregistrer les interactions des clients avec les articles & 2 & 2 & \cellcolor{green!50} 4 \\ 
Data Scientist & Données articles mal labellisées & Petit nombre de bonnes images & 3 & 3 & \cellcolor{yellow!75} 9 & Utiliser une technique de data augmentation & Regrouper des centaines d’images par catégories & 2 & 2 & \cellcolor{green!50} 4 \\ 
Data Scientist & RGPD trop contraignant & Les photos doivent être anonymisés & 3 & 3 & \cellcolor{yellow!75} 6 & Effacer les métadonnées des photos & Flouter les visages & 2 & 1 & \cellcolor{green!50} 2 \\ 
"""

In [24]:
d = data.replace('\\cellcolor{gray!50}', '')\
.replace('\\cellcolor{gray!15}', '')\
.replace('\\cellcolor{gray!25}', '')\
.replace('\\cellcolor{red!75}', '')\
.replace('\\cellcolor{red!50}', '')\
.replace('\\cellcolor{yellow!50}', '')\
.replace('\\cellcolor{green!50}', '')\
.replace('\\cellcolor{yellow!75}', '')\
.replace("\\parbox{1.5cm}{", " ")\
.replace("}", " ")\
.replace('\bf', ' ')\
.replace('\\dots', '...')\
.replace('\\hline', '').replace('\n',' ')\
.replace('\t', ' ').replace('\\', ' \n ').replace('&', ' \t ')


In [25]:
d.encode('utf-8').decode()

'      Resp.   \t    Accident  \t    Probables Causes  \t     Initial Risk Probabilité   \t    Sévérité  \t    Risk Level  \t    Actions Preventives \t    Reparations  \t     Residual Risk Probabilité   \t   Sévérité  \t    Risk Level  \n    Codeur   \t  Usurpation d’identité  \t  Stockage en clair du password \t  3  \t  4  \t   12  \t  Fonction de dérivation, graine et salage du mot de passage  \t  Alerte client pour toute nouvelle connexion IP  \t  1  \t  2  \t   2  \n     Codeur   \t  Injection  \t  Données hostiles  \t  4  \t  4  \t   16  \t  Échappement des chars spéciaux, utilisation de LIMIT en SQL  \t  Stocker données nécessaires, seulement  \t  3  \t  4  \t   12  \n     Codeur   \t  Déni de service  \t  Fichier accessible par tous via lien  \t  5  \t  2  \t   10  \t   Tester la performance E/S  \t  Réarchitecturer, optimiser, mettre en cache  \t  4  \t  2  \t   8  \n   Data Scientist  \t  Intégrité des données  \t  Data sérialisées non sécurisées  \t  3  \t  4  \t   12  \t   C

In [26]:
with open('risk_prevention.csv', 'w') as f:
    f.writelines(d.encode('utf-8').decode())

In [27]:
pd.read_csv('risk_prevention.csv', sep='\t').to_excel('risk_prevention.xlsx')

In [28]:
from openpyxl import load_workbook
wb_backlog = load_workbook(filename = 'risk_prevention.xlsx')
ws_backlog = wb_backlog['Sheet1']
print(ws_backlog['B3'].value)

     Codeur   


In [29]:
greenLightFill = PatternFill(start_color='80ff80',
                   end_color='80ff80',
                   fill_type='solid')
yellowFill = PatternFill(start_color='fff980',
                   end_color='fff980',
                   fill_type='solid')
redLightFill = PatternFill(start_color='ff8080',
                   end_color='ff8080',
                   fill_type='solid')
redDarkFill = PatternFill(start_color='ff4040',
                   end_color='ff4040',
                   fill_type='solid')

In [30]:
ws_backlog.delete_cols(1, 1)

In [31]:
for l in ['F', 'K']:
    for cell in ws_backlog[ l + '2:' + l + '13']:
        text = str(cell[0].value).rstrip().lstrip()
        for x in range(len(cell)):
            if float(text) <= 5. :
                cell[x].fill = greenLightFill
            elif (float(text) > 5.) & (float(text) <= 10.) :
                cell[x].fill = yellowFill
            elif (float(text) > 10.) & (float(text) <= 15.) :
                cell[x].fill = redLightFill
            elif float(text) > 15. :
                cell[x].fill = redDarkFill

In [32]:
ws_backlog.title = "Risk_prevention"
wb_backlog.save(filename = 'risk_prevention.xlsx')